In [1]:
from ast import literal_eval
import urllib, json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression


%matplotlib inline

In [4]:
households = pd.read_csv('Housing_survey/household.csv')

In [6]:
ny_households = pd.read_csv('Housing_characteristics/ACS_15_5YR_S2504_with_ann.csv')

/Users/mariamendieta/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
ny_households.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC01_EST_VC03,...,HC02_EST_VC52,HC02_MOE_VC52,HC03_EST_VC52,HC03_MOE_VC52,HC01_EST_VC53,HC01_MOE_VC53,HC02_EST_VC53,HC02_MOE_VC53,HC03_EST_VC53,HC03_MOE_VC53
0,Id,Id2,Geography,Occupied housing units; Estimate; Occupied hou...,Occupied housing units; Margin of Error; Occup...,Owner-occupied housing units; Estimate; Occupi...,Owner-occupied housing units; Margin of Error;...,Renter-occupied housing units; Estimate; Occup...,Renter-occupied housing units; Margin of Error...,Occupied housing units; Estimate; UNITS IN STR...,...,Owner-occupied housing units; Estimate; HOUSE ...,Owner-occupied housing units; Margin of Error;...,Renter-occupied housing units; Estimate; HOUSE...,Renter-occupied housing units; Margin of Error...,Occupied housing units; Estimate; HOUSE HEATIN...,Occupied housing units; Margin of Error; HOUSE...,Owner-occupied housing units; Estimate; HOUSE ...,Owner-occupied housing units; Margin of Error;...,Renter-occupied housing units; Estimate; HOUSE...,Renter-occupied housing units; Margin of Error...
1,1400000US36001000100,36001000100,"Census Tract 1, Albany County, New York",815,73,246,64,569,93,21.0,...,0.0,11.5,0.0,5.1,0.0,3.6,0.0,11.5,0.0,5.1
2,1400000US36001000200,36001000200,"Census Tract 2, Albany County, New York",2006,228,525,127,1481,189,7.4,...,0.0,5.6,0.0,2.0,1.5,1.6,0.0,5.6,2.0,2.1
3,1400000US36001000300,36001000300,"Census Tract 3, Albany County, New York",2311,156,906,180,1405,203,31.6,...,0.0,3.3,0.0,2.1,2.3,2.2,0.0,3.3,3.8,3.6
4,1400000US36001000401,36001000401,"Census Tract 4.01, Albany County, New York",1037,82,617,76,420,92,23.9,...,1.0,1.7,1.4,2.2,0.5,1.0,0.0,4.7,1.2,2.3


In [18]:
ny_households.drop([0], inplace=True)

In [21]:
ny_households.shape

(4918, 231)

In [15]:
dct = {'DP04_0001E': 'Housing_Units',
       'DP04_0001PE':'Percent_Housing', 
       'DP04_0002E': 'Occupied_units',
       'DP04_0002PE': 'Occupied_units_percent',
       'DP04_0005E': 'Rental_vacancy_rate',
       'DP04_0006E':'Units_in_structure',
       'DP04_0006PE':'Units_in_structure_percent',
       'DP04_0007E': '1_unit_detached',
       'DP04_0007PE':'1_unit_detached_percent',
       'DP04_0008E': '1_unit_attached',
       'DP04_0009E': '2_units',
       'DP04_0010E': '3_4_units',
       'DP04_0011E': '5_9_units',
       'DP04_0012E':  '10_19_units',
       'DP04_0012E':  '10_19_units',
       'DP04_0027E':'Total_housing_units_Rooms',
       'DP04_0028E':'1_room',
       'DP04_0029E':'2_rooms',
       'DP04_0030E':'3_rooms',
       'DP04_0031E':'4_rooms',
       'DP04_0032E':'5_rooms',
       'DP04_0033E':'6_rooms',
       'DP04_0034E':'7_rooms',
       'DP04_0035E':'8_rooms',
       'DP04_0036E':'9_more_rooms',
       'DP04_0037E':'median_rooms',
       'DP04_0062E':'Occupied_units_heating',
       'DP04_0063E':'Heating_gas',
       'DP04_0063E':'Heating_electricity',
       'DP04_0063E':'Heating_fuelkero',
       'DP04_0063E':'Heating_coal_coke',
       'DP04_0063E':'Heating_wood',
       'DP04_0063E':'Heating_solar',
       'DP04_0063E':'Heating_other',
       'DP04_0063E':'Heating_no'
      }

lst = dct.keys()
var = ','.join(lst)
lst_names = dct.values()

#URL  http://api.census.gov/data/2015/acs5/profile?  State = 36 (New York), Level = Tract
key = 'ff3aee5f79063ffd4c2fa0b7912d2b83f974b2f8'

hous_acs_url = 'https://api.census.gov/data/2015/acs5/profile?get={},NAME&for=tract:*&in=state:36&key={}'.format(var, key)

In [16]:
response = requests.get(hous_acs_url)
#print(response.text)
request_content_list = literal_eval(response.text)
headers = request_content_list.pop(0)
acs = pd.DataFrame(request_content_list, columns=headers)
acs.head()

,DP04_0002E,DP04_0011E,DP04_0010E,DP04_0062E,DP04_0001PE,DP04_0005E,DP04_0006E,DP04_0035E,DP04_0007E,DP04_0027E,...,DP04_0029E,DP04_0031E,DP04_0012E,DP04_0037E,DP04_0006PE,DP04_0007PE,NAME,state,county,tract
0,815,17,127,815,953,6.8,953,27,180,953,...,0,200,27,5.2,953,18.9,"Census Tract 1, Albany County, New York",36,001,000100
1,2006,150,728,2006,2722,15.0,2722,97,149,2722,...,42,746,248,4.6,2722,5.5,"Census Tract 2, Albany County, New York",36,001,000200
2,2311,57,562,2311,2695,2.2,2695,227,731,2695,...,99,489,0,5.1,2695,27.1,"Census Tract 3, Albany County, New York",36,001,000300
3,1037,152,83,1037,1132,8.1,1132,31,248,1132,...,73,418,10,4.4,1132,21.9,"Census Tract 4.01, Albany County, New York",36,001,000401
4,2300,209,417,2300,2559,9.9,2559,323,1175,2559,...,32,521,88,5.7,2559,45.9,"Census Tract 4.03, Albany County, New York",36,001,000403


In [17]:
#Renaming features
acs.rename(columns=dct,inplace=True)
# Creating full tract number
acs["GEOID10"] = acs["state"].map(str) + acs["county"].map(str) + acs["tract"].map(str)
# Changing type to numeric
for fea  in lst_names:
    acs[fea] = pd.to_numeric(acs[fea], errors='coerce')
acs["GEOID10"] = pd.to_numeric(acs['GEOID10'], errors='coerce')


In [11]:
for fea  in lst_names:
    print type(acs[fea])
#acs_hous["GEOID10"] = pd.to_numeric(acs_hous['GEOID10'], errors='coerce')


<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [12]:
lst_names

['Occupied_units',
 '5_9_units',
 '3_4_units',
 'Occupied_units_heating',
 'Percent_Housing',
 'Rental_vacancy_rate',
 'Units_in_structure',
 '8_rooms',
 '1_unit_detached',
 'Total_housing_units_Rooms',
 'Occupied_units_percent',
 '6_rooms',
 '5_rooms',
 'Housing_Units',
 '7_rooms',
 '1_unit_attached',
 'Heating_no',
 '2_units',
 '3_rooms',
 '1_room',
 '9_more_rooms',
 '2_rooms',
 '4_rooms',
 '10_19_units',
 'median_rooms',
 'Units_in_structure_percent',
 '1_unit_detached']

In [14]:
acs['1_unit_detached']

,1_unit_detached,1_unit_detached
0,180,18.9
1,149,5.5
2,731,27.1
3,248,21.9
4,1175,45.9
5,4,19.0
6,219,12.1
7,233,23.5
8,189,11.1
9,251,12.6
